# Caso Práctico - ENGIE

*Jose Enrique Zafra Mena*

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl

## Carga y preparación de Datos


In [2]:
facturas_df=pd.read_excel('facturas.xlsx')
mapping_df=pd.read_excel('Mapping.xlsx')

# renombramos los Deals
CTV_df=pd.read_excel('JVLNG_CTV_STOK_MARZO_2024_05_23_14_12.xlsx')
IAC_df=pd.read_excel('MEDG2_IAC_GETRA_MARZO_2024_05_23_14_12.xlsx')
AOC2_df=pd.read_excel('SUGS2_AOC_STOK_MARZO_2024_05_23_14_12.xlsx')
AOC_df=pd.read_excel('SUGST_AOC_STOK_MARZO_2024_05_23_14_12.xlsx')


In [3]:
# verificamos las primeras filas de cada dataframe
print("Facturas AGIkey:")
print(facturas_df.head())

print("\nMapping:")
print(mapping_df.head())

print("\nCTV:")
print(CTV_df.head())

print("\nIAC:")
print(IAC_df.head())

print("\nAOC2:")
print(AOC2_df.head())

print("\nAOC:")
print(AOC_df.head())

Facturas AGIkey:
   NumeroFactura       Emisor       RazonSocialEmisor  DepEmisor     Receptor  \
0     2024264672  ESA86484292         ENAGAS GTS, SAU       60.0  ESB82508441   
1     2024264671  ESA86484292         ENAGAS GTS, SAU       60.0  ESB82508441   
2     2324004231  ESA86484334  ENAGAS TRANSPORTE, SAU        NaN  ESB82508441   
3     2024164592  ESA86484292         ENAGAS GTS, SAU       50.0  ESB82508441   
4     2024164591  ESA86484292         ENAGAS GTS, SAU       50.0  ESB82508441   

  RazonSocialReceptor  DepReceptor FechaFactura    Importe Moneda    Estado  \
0    ENGIE ESPAÑA SLU          NaN       4/3/24  123195.13    EUR  Recibida   
1    ENGIE ESPAÑA SLU          NaN       4/3/24  111963.62    EUR  Recibida   
2    ENGIE ESPAÑA SLU          NaN       6/3/24  503763.47    EUR  Recibida   
3    ENGIE ESPAÑA SLU          NaN       4/3/24    1375.12    EUR  Recibida   
4    ENGIE ESPAÑA SLU          NaN       4/3/24    1787.43    EUR  Recibida   

  FechaEstado FechaRe

In [61]:
# verificamos las columnas de cada dataframe
print("\nColumnas de Facturas:")
print(facturas_df.columns)

print("\nColumnas de Mapping:")
print(mapping_df.columns)

print("\nColumnas de CTV:")
print(CTV_df.columns)
 
print("\nColumnas de IAC:")
print(IAC_df.columns)

print("\nColumnas de AOC2:")
print(AOC2_df.columns)

print("\nColumnas de AOC:")
print(AOC_df.columns)



Columnas de Facturas:
Index(['NumeroFactura', 'Emisor', 'RazonSocialEmisor', 'DepEmisor', 'Receptor',
       'RazonSocialReceptor', 'DepReceptor', 'FechaFactura', 'Importe',
       'Moneda', 'Estado', 'FechaEstado', 'FechaRegistro', 'Destino',
       'Contrato', 'Origen', 'ServicioFacturado', '¿Verificada por BO?',
       'Observaciones'],
      dtype='object')

Columnas de Mapping:
Index(['Origen', 'Servicio facturado', 'Portfolio', 'Commodity', 'DealType'], dtype='object')

Columnas de CTV:
Index(['Unnamed: 0', 'AggregatedKey', 'ApplicationPeriod', 'BookingSource',
       'BOValidated', 'BOValidationDate', 'BOValidator', 'CancelsDealId',
       'Capacity', 'CapacityFrequency',
       ...
       'ExerciseDate', 'ExerciseType', 'Nature', 'OptionNature', 'OptionType',
       'PeriodInfoCollection', 'Script', 'TemplateId', 'TemplateScript',
       'OrderId'],
      dtype='object', length=108)

Columnas de IAC:
Index(['Unnamed: 0', 'AggregatedKey', 'ApplicationPeriod', 'AuctionType',
   

Vamos a renombrar columnas comunes,

In [57]:
facturas_df.rename(columns={'ServicioFacturado': 'Servicio Facturado'}, inplace=True)
mapping_df.rename(columns={'Servicio facturado': 'Servicio Facturado'}, inplace=True)

## Transformación de los datos

Vamos a conectar el Mapping con cada Deal, ya que queremos comparar con las facturas.

Para conectar Mapping con CTV usamos Commodity==CTV

Para conectar Mapping con IAC usamos Commodity==IAC

Para conectar Mapping con AOC usamos Commodity==AOC & Portofolio: SUGST

Para conectar Mapping con AOC2 usamos Commodity==AOC & Portofolio: SUGS2


In [96]:
# para poder unir cada Deal con Mapping individualmente, añadimos las columnas Portofolio y Commodity a cada deal
# y le asignamos su nombre correspondiente

CTV_df['Portfolio']='JVLNG'
CTV_df['Commodity']='CTV'


IAC_df['Portfolio']='MEDG2'
IAC_df['Commodity']='IAC'

AOC_df['Portfolio']='SUGST'
AOC_df['Commodity']='AOC'

AOC2_df['Portfolio']='SUGS2'
AOC2_df['Commodity']='AOC'


In [101]:
# ahora unimos cada uno con mapping

CTV_mapped_df = pd.merge(CTV_df,mapping_df,on='Commodity',how='inner')
IAC_mapped_df = pd.merge(IAC_df,mapping_df,on='Commodity',how='inner')
AOC_mapped_df = pd.merge(AOC_df, mapping_df, on=['Commodity', 'Portfolio'], how='inner')
AOC2_mapped_df = pd.merge(AOC2_df,mapping_df,on=['Commodity', 'Portfolio'],how='inner')

#print(CTV_mapped_df.head())
print(IAC_mapped_df.shape)

(2, 92)


## Comparar con Facturas

In [102]:
common_columns_CTV=facturas_df.columns.intersection(CTV_mapped_df.columns).intersection(CTV_mapped_df.columns)

comparison = facturas_df[common_columns_CTV] == CTV_mapped_df[common_columns_CTV]

differences = comparison[~comparison.all(axis=1)]

print(differences.head())

ValueError: Can only compare identically-labeled (both index and columns) DataFrame objects

## Propuesta de Valor